In [7]:

import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import xml.etree.ElementTree as ET
import datetime
import os

# === Config ===
BASE_URL = "https://www.bog.gov.sa"
HANDLER_URL = f"{BASE_URL}/_layouts/15/BOG/handlers/BOGBlogHandler.ashx"
HEADERS = {"User-Agent": "Mozilla/5.0"}

requests.packages.urllib3.disable_warnings()


# -------------------------------
# STEP 1: Fetch all collections
# -------------------------------
def fetch_page_links(page_index):
    url = f"{BASE_URL}/ScientificContent/JudicialBlogs/Pages/JudgmentsDefault.aspx?PageIndex={page_index}"
    print(f"📂 Fetching page: {url}")
    resp = requests.get(url, headers=HEADERS, verify=False)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    links = []
    for a in soup.select("a.blog-ver-link"):
        title = a.get_text(strip=True)
        href = a.get("href")
        if title and href:
            links.append({"title": title, "url": href})
    return links


def get_all_collections():
    all_collections = []
    for i in range(1, 3):  # Only 2 pages exist
        page_links = fetch_page_links(i)
        if not page_links:
            break
        all_collections.extend(page_links)

    # Deduplicate by title
    seen = set()
    unique_collections = []
    for entry in all_collections:
        if entry["title"] not in seen:
            seen.add(entry["title"])
            unique_collections.append(entry)

    return unique_collections


# -------------------------------
# STEP 2: Crawl PDFs inside collection
# -------------------------------
def call_api(op, siteUrl, parent, libname="المستندات", contentType="مجلد ديوان المظالم", rowLimit=500):
    params = {
        "op": op,
        "siteUrl": siteUrl,
        "parent": parent,
        "libname": libname,
    }
    if op == "LoadItems":
        params["rowLimit"] = str(rowLimit)
        params["viewName"] = "كافة المستندات"
        params["folderContentType"] = contentType
    else:
        params["contentType"] = contentType

    encoded_params = "&".join([f"{k}={quote(v)}" for k, v in params.items()])
    url = f"{HANDLER_URL}?{encoded_params}"

    print(f"📂 Fetching: {parent} ({op})")
    r = requests.get(url, verify=False)
    r.raise_for_status()
    return ET.fromstring(r.text)


def crawl_folder(siteUrl, parent, collected, path=[], depth=0, max_depth=15):
    if depth > max_depth:
        print("⚠️ Max recursion depth reached, stopping further crawling.")
        return

    try:
        xml_root = call_api("LoadSubLevel", siteUrl, parent)
        items = xml_root.findall("Item")
    except ET.ParseError:
        print("⚠️ XML ParseError, skipping this branch.")
        return

    if items:
        for item in items:
            title = item.attrib.get("Title") or item.findtext("Title")
            file_ref = item.attrib.get("FileLeafRef")
            if not file_ref:
                continue
            new_parent = f"{parent}/{file_ref}"
            crawl_folder(siteUrl, new_parent, collected, path + [file_ref], depth + 1, max_depth)
    else:
        try:
            xml_root = call_api("LoadItems", siteUrl, parent)
            items = xml_root.findall("Item")
        except ET.ParseError:
            print("⚠️ XML ParseError in LoadItems, skipping.")
            return

        for item in items:
            title = item.attrib.get("Title")
            filename = item.attrib.get("FileLeafRef")
            docicon = item.attrib.get("DocIcon")
            if docicon and docicon.lower() == "pdf":
                pdf_url = f"{BASE_URL}{parent}/{quote(filename)}"
                collected.append({"path": " > ".join(path + [title]), "url": pdf_url})
                print(f"   ✅ Found PDF: {pdf_url}")


# -------------------------------
# MAIN
# -------------------------------
def main():
    collections = get_all_collections()

    print(f"\n=== Found {len(collections)} unique collections ===")
    for idx, col in enumerate(collections, 1):
        print(f"[{idx}] {col['title']} -> {col['url']}")

    chosen = input("\nEnter collection indexes to crawl (comma-separated): ").strip()
    indexes = [int(x) for x in chosen.split(",") if x.strip().isdigit()]

    # Create dated output folder
    today_str = datetime.datetime.now().strftime("%d.%m.%Y")
    out_dir = f"BOG PDF Crawled {today_str}"
    os.makedirs(out_dir, exist_ok=True)

    for idx in indexes:
        if idx < 1 or idx > len(collections):
            print(f"⚠️ Skipping invalid index: {idx}")
            continue

        col = collections[idx - 1]
        siteUrl = "/" + "/".join(col["url"].split("/", 3)[3:])
        if siteUrl.endswith("/Pages/default.aspx"):
            siteUrl = siteUrl.replace("/Pages/default.aspx", "")

        parent = f"{siteUrl}/Documents"

        print(f"\n🚀 Crawling collection [{idx}] {col['title']} ({col['url']})")
        collected = []
        crawl_folder(siteUrl, parent, collected, path=["Documents"])

        # Save per collection
        safe_title = col['title'].replace(" ", "_").replace("/", "-")
        out_file = os.path.join(out_dir, f"{safe_title}.txt")
        with open(out_file, "w", encoding="utf-8") as f:
            for entry in collected:
                f.write(f"{entry['path']}\n{entry['url']}\n\n")

        print(f"=== Total PDFs found in '{col['title']}': {len(collected)} ===")
        print(f"📂 Saved to {out_file}")

    print(f"\n✅ All results saved inside folder: {out_dir}")


if __name__ == "__main__":
    main()


📂 Fetching page: https://www.bog.gov.sa/ScientificContent/JudicialBlogs/Pages/JudgmentsDefault.aspx?PageIndex=1
📂 Fetching page: https://www.bog.gov.sa/ScientificContent/JudicialBlogs/Pages/JudgmentsDefault.aspx?PageIndex=2

=== Found 20 unique collections ===
[1] مجموعة الأحكام والمبادئ الإدارية للأعوام ١٤٠٢-١٤٢٦ هـ -> https://www.bog.gov.sa/ScientificContent/JudicialBlogs/AA1402-1426/Pages/default.aspx
[2] مجموعة الأحكام والمبادئ الجزائية للأعوام ١٤٠٢هـ-١٤٢٧هـ -> https://www.bog.gov.sa/ScientificContent/JudicialBlogs/1402-1427/Pages/default.aspx
[3] مجموعة الأحكام والمبادئ التجارية للأعوام ١٤٠٨ -١٤٢٣ هـ -> https://www.bog.gov.sa/ScientificContent/JudicialBlogs/1408-1423/Pages/default.aspx
[4] مجموعة الأحكام والمبادئ التجارية للأعوام ١٤٢٤ -١٤٢٧ هـ -> https://www.bog.gov.sa/ScientificContent/JudicialBlogs/1424-1427/Pages/default.aspx
[5] مجموعة الأحكام والمبادئ الإدارية لعام ١٤٢٧ هـ -> https://www.bog.gov.sa/ScientificContent/JudicialBlogs/Blog1427/Pages/default.aspx
[6] مجموعة الأحكام

In [ ]:
# Stage 2 loads and lists all “collection_name.txt” collection files created in Stage 1 and prepares the download process.

import os
import requests
import datetime

# Disable SSL warnings
requests.packages.urllib3.disable_warnings()


def list_collections_folder():
    today_str = datetime.datetime.now().strftime("%d.%m.%Y")
    base_folder = f"BOG PDF Crawled {today_str}"

    if not os.path.exists(base_folder):
        print(f"❌ Folder '{base_folder}' not found. Run the crawler first.")
        return None, []

    txt_files = [f for f in os.listdir(base_folder) if f.endswith(".txt")]
    return base_folder, txt_files


def safe_filename(name, max_len=80):
    """Clean filename (Arabic supported), truncate if too long."""
    name = name.replace("/", "-").replace("\\", "-").replace(":", "-").replace("?", "")
    if len(name) > max_len:
        name = name[:max_len] + "_trunc"
    return name + ".pdf"


def download_pdfs_from_txt(base_folder, txt_file):
    collection_name = os.path.splitext(txt_file)[0]
    collection_path = os.path.join(base_folder, txt_file)

    pdf_folder = os.path.join(base_folder, collection_name)
    os.makedirs(pdf_folder, exist_ok=True)

    entries = []
    with open(collection_path, "r", encoding="utf-8") as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]
        for i in range(0, len(lines), 2):
            if i + 1 < len(lines):
                path_line, url = lines[i], lines[i+1]
                if url.startswith("http") and url.endswith(".pdf"):
                    # filename from last part of the path (after last ">")
                    last_part = path_line.split(">")[-1].strip()
                    filename = safe_filename(last_part)
                    entries.append((url, filename))

    print(f"\n📂 Found {len(entries)} PDFs in '{txt_file}'. Starting download...")

    for i, (url, filename) in enumerate(entries, 1):
        save_path = os.path.join(pdf_folder, filename)

        if os.path.exists(save_path):
            print(f"   ⏭️ Skipping (already exists): {filename}")
            continue

        try:
            r = requests.get(url, verify=False, timeout=60)
            r.raise_for_status()
            with open(save_path, "wb") as f:
                f.write(r.content)
            print(f"   ✅ [{i}/{len(entries)}] Downloaded: {filename}")
        except Exception as e:
            print(f"   ❌ Failed: {filename} ({e})")


def main():
    base_folder, txt_files = list_collections_folder()
    if not txt_files:
        return

    print("\nAvailable collections to download:")
    for idx, f in enumerate(txt_files, 1):
        print(f"[{idx}] {f}")

    chosen = input("\nEnter collection indexes to download (comma-separated): ").strip()
    indexes = [int(x) for x in chosen.split(",") if x.strip().isdigit()]

    for idx in indexes:
        if idx < 1 or idx > len(txt_files):
            print(f"⚠️ Invalid index: {idx}")
            continue
        txt_file = txt_files[idx - 1]
        download_pdfs_from_txt(base_folder, txt_file)

    print(f"\n✅ Downloads completed. Check inside '{base_folder}'.")


if __name__ == "__main__":
    main()
